# 05-3 트리의 앙상블

## 랜덤 포레스트

### 랜덤 포레스트 훈련

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 와인 데이터셋을 판다스로 불러오고 훈련 세트와 테스트 세트로 나s눔
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# cross_validate() 함수를 사용해 교차 검증을 수행
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# 출력된 결과를 보면 훈련 세트에 다소 과대적합된 것 같음
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
# 두 번째 특성인 당도의 중요도가 감소하고 알코돌 도수와 pH 특성의 중요도가 조금 상승한 것을 볼 수 있음
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
# 부트스트랩 샘플에 포함되지 않고 남은 샘플을 OOB(out of bag)샘플
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

### 엑스트라 트리

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
# 엑스트라 트리의 교차 검증 점수 확인
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
# 특성 중요도
# 결정 트리보다 당도에 대한 의존성이 작음
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

### 그레이디언트 부스팅

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
# GradientBoostringClassifier를 사용해 와인 데이터셋의 교차 검증 점수를 확인
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
# 결정 트리 개수를 500개로 5배 늘렸지만 과대적합을 잘 억제하는 것을 볼 수 있음
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
# 특성 중요도를 제공
# 랜덤 포레스트보다 일부 특성인 당도에 더 집중함을 볼 수 있음
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그레이디언트 부스팅

### 히스토그램 기반 그레이디언트 부스팅

In [13]:
# HistGradientBoostingClassifier는 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


### Permutation Importance

In [14]:
from sklearn.inspection import permutation_importance

# 히스토그램 기반 그레이디언트 부스팅 모델을 훈련한 후
# 훈련 세트에서 특성 중요도를 계산하기 위해 n_repeats 매개변수로 랜덤하게 섞을 횟수를 지정
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
# 특성 중요도, 평균, 표준 편차를 담고 있음
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [15]:
# 테스트 세트에서 특성 중요도를 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
# 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [16]:
# HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능을 최종적으로 확인
hgb.score(test_input, test_target)

0.8723076923076923

### XGBoost vs LightGBM

In [17]:
# XGBoost를 사용해 와인 데이터의 교차 검증 점수를 확인
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [18]:
# LightGBM를 사용해 와인 데이터의 교차 검증 점수를 확인
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
